In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from method import IV
from simulator import Simulator
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import pandas as pd
import numpy as np
import quantities as pq
import seaborn as sns
from exana.statistics import plot_xcorr, permutation_resampling
from tools import savefig, fix_figure, coef_var, corrcoef, despine, set_style
import copy
from IPython.display import display
import statsmodels.api as sm

/home/mikkel/.virtualenvs/main/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [47]:
params = {}
for i in [1,2,3]:
    d = np.load('results/params_{}.npz'.format(i))['data'][()]
    params['model {}'.format(i)] = d['params']
del d

In [48]:
params['model 1'].keys()

dict_keys(['V_reset', 'msd', 'stim_amp_ex', 'verbose', 'N_rec_state_ex', 'C_m', 'E_L', 'NA', 'rate_in', 'J_high_in', 'stim_amp_in', 'N_in', 'S', 'num_threads', 'stim_N_ex', 'tau_syn_ex', 'C_ex', 'rate_p', 'J_high_ex', 'stim_duration', 'p_var', 'r', 'res', 'stim_dist', 'J', 'tau_m', 'stim_nodes_in', 'J_ex', 't_ref', 'N_ex', 'n', 'data_path', 'stim_N_in', 'status', 'delay', 'eps', 'C_in', 'tau_syn_in', 'stim_max_period', 'N_rec_state_in', 'J_low', 'stim_nodes_ex', 'stim_period', 'J_high', 'rate_ex', 'V_m', 'eta', 'stim_N', 'N_rec_spike_in', 'J_in', 'N_neurons', 'J_low_in', 'J_low_ex', 'fname', 'g', 'V_th', 'density', 'N_rec_spike_ex'])

In [80]:
interest = ['N_neurons', 'res', 'N_ex', 'N_in', 'eta', 'rate_p',
            'V_reset', 'V_m', 'E_L', 't_ref', 'tau_m','V_th',  'C_m',
            'J', 'J_high', 'J_low', 'p_var', 'g', 'tau_syn_in', 'tau_syn_ex', 'delay', 'eps', 'C_ex', 'C_in',
            'J_in', 'J_ex', 'J_high_ex', 'J_low_ex', 'J_high_in', 'J_low_in',
            'stim_N_in', 'stim_N_ex', 'stim_amp_in', 'stim_amp_ex', 'stim_duration', 'stim_period', 'stim_max_period',
            'rate_in', 'rate_ex',
            'density', 'S', 'NA', 'r', 'n']
units = {
    'rate_p': 'Hz', 
    'V_reset': 'mV', 
    'V_m': 'mV', 
    'E_L': 'mV', 
    't_ref': 'ms',
    'tau_m': 'ms',
    'V_th': 'mV',
    'C_m': 'pF',
    'J': 'mV',
    'J_high': 'mV',
    'J_low': 'mV',
    'p_var': 'mV$^2$',
    'tau_syn_in': 'ms',
    'tau_syn_ex': 'ms',
    'delay': 'ms',
    'J_in': 'pA',
    'J_ex': 'pA',
    'J_high_ex': 'pA',
    'J_low_ex': 'pA',
    'J_high_in': 'pA',
    'J_low_in': 'pA',
    'stim_amp_in': 'pA',
    'stim_amp_ex': 'pA',
    'stim_duration': 'ms',
    'stim_period': 'ms',
    'stim_max_period': 'ms',
    'rate_in': 'Hz',
    'rate_ex': 'Hz',
    'density': 'Nmm$^{-3}$',
    'S': 'mm$^{-1}$', 
    'r': '$\mu$m'
        }
units = {k: units[k] if k in units else '' for k in params['model 1'].keys()}
params['units'] = units
df = pd.DataFrame(params, columns=['model 1', 'model 2', 'model 3', 'units']).loc[interest, :]

In [81]:
cmds = ['tau']
def fix_underscore(s):
    subs = s.split('_')
    if len(subs) in [2,3]:
        if subs[0] in cmds:
            subs[0] = '\\' + subs[0]
        label = ''.join(['$'] + [subs[0]] +
                        [t + '{' + ss + '}' for ss, t in zip(subs[1:], ['_', '^'])] + ['$'])
    elif len(subs) == 1:
        label = '$' + s + '$'
    else:
        raise ValueError('Found {} underscores'.format(len(subs)))
    return label

In [82]:
rename_idx = {k: fix_underscore(k) for k in df.index}
# rename_idx = {'N_neurons': 'N_{neurons}'}
df = df.rename(index=rename_idx).astype(object)
df

,model 1,model 2,model 3,units
$N_{neurons}$,1250,1250,1250,
$res$,0.1,0.1,0.1,
$N_{ex}$,1000,1000,1000,
$N_{in}$,250,250,250,
$eta$,0.9,0.9,0.9,
$rate_{p}$,3694.26,3694.26,3694.26,Hz
$V_{reset}$,0,0,0,mV
$V_{m}$,0,0,0,mV
$E_{L}$,0,0,0,mV
$t_{ref}$,2,2,2,ms


In [83]:
df.to_latex('manuscript/param_table.tex', escape=False)